In [124]:
#! pip install kaggle

In [125]:
# configuring the path of Kaggle.json file
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json


In [126]:
# import kagglehub
# import os
# import pandas as pd
# import shutil

# # Step 1: Download dataset
# path = kagglehub.dataset_download("kritanjalijain/amazon-reviews")
# print("Path to dataset files:", path)

# # Step 2: List files in the dataset folder
# files = os.listdir(path)
# print("Files in dataset folder:", files)

# # Step 3: Find the CSV file (modify if needed)
# csv_file = None
# for file in files:
#     if file.endswith(".csv"):
#         csv_file = os.path.join(path, file)
#         break

# # Step 4: If CSV is found, load it into Pandas
# if csv_file:
#     df = pd.read_csv(csv_file)
#     print("CSV file loaded successfully!")
#     print(df.head())  # Show first few rows

#     # Step 5: Move the CSV to Google Colab's /content/ directory for easy access
#     destination = "/content/amazon_reviews.csv"
#     shutil.copy(csv_file, destination)
#     print(f"File moved to: {destination}")

#     # Now you can load it directly with:
#     df = pd.read_csv(destination)
# else:
#     print("No CSV file found in the dataset folder.")


## IMPORT LIBRARIES


In [128]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer


## LOAD DATASET

In [130]:
data=pd.read_csv("amazon_reviews.csv")
data.head()

,2,Great CD,"My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I'm in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life's hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this, no matter black, white, young, old, male, female EVERYBODY says one thing ""Who was that singing ?"""
0,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
1,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
2,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
3,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...
4,1,DVD Player crapped out after one year,I also began having the incorrect disc problem...


In [131]:
data.columns = ["Rating", "Title", "Review"]
data.head()

,Rating,Title,Review
0,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
1,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
2,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
3,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...
4,1,DVD Player crapped out after one year,I also began having the incorrect disc problem...


In [132]:
data["Review"] = data["Title"] + " " + data["Review"]
data = data[['Review', 'Rating']]
data.head()

,Review,Rating
0,One of the best game music soundtracks - for a...,2
1,Batteries died within a year ... I bought this...,1
2,"works fine, but Maha Energy is better Check ou...",2
3,Great for the non-audiophile Reviewed quite a ...,2
4,DVD Player crapped out after one year I also b...,1


In [133]:
def sentiment(label):
    if label==1:
        return "Negative"
    elif label==2:
        return "Positive"
    else:
        return "Neutral"

In [134]:
data['Sentiment'] = data["Rating"].apply(sentiment)
data.head(10)

,Review,Rating,Sentiment
0,One of the best game music soundtracks - for a...,2,Positive
1,Batteries died within a year ... I bought this...,1,Negative
2,"works fine, but Maha Energy is better Check ou...",2,Positive
3,Great for the non-audiophile Reviewed quite a ...,2,Positive
4,DVD Player crapped out after one year I also b...,1,Negative
5,"Incorrect Disc I love the style of this, but a...",1,Negative
6,DVD menu select problems I cannot scroll throu...,1,Negative
7,Unique Weird Orientalia from the 1930's Exotic...,2,Positive
8,"Not an ""ultimate guide"" Firstly,I enjoyed the ...",1,Negative
9,Great book for travelling Europe I currently l...,2,Positive


In [135]:
data.value_counts("Sentiment")


Sentiment
Negative    200000
Positive    199999
Name: count, dtype: int64

In [136]:
data.isna().sum()
data = data.fillna(' ')
data.isna().sum()

Review       0
Rating       0
Sentiment    0
dtype: int64

## Sampling the data

In [138]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.97)
for train_index, test_index in split.split(data, data["Sentiment"]):
    strat_data = data.reindex(train_index)

In [139]:
strat_data.head(10)


,Review,Rating,Sentiment
149920,"Scatter-brained references every page, little ...",1,Negative
128874,"Complete Crap A novel idea, however they just ...",1,Negative
159208,Splendid Reading The problem I find with relig...,2,Positive
99436,Shouldn't a CARS game be about racing? I purch...,1,Negative
354859,Planet Earth (blu-ray) - Out of this world!!! ...,2,Positive
101910,Doesn't hold up long We have a small car (hond...,1,Negative
349136,Fantastic! Pricey but worth every penny! Perso...,2,Positive
48541,"Been there, done that, only not as good Can't ...",1,Negative
96942,Paint comes off VERY easily! I am sorry to say...,1,Negative
103884,very unstable to put together At first I was m...,1,Negative


In [140]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download required NLTK resources (Run this ONCE before calling preprocess_text)
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

def preprocess_text(text):
    """
    Preprocesses the given text by:
    - Converting to lowercase
    - Removing URLs, special characters, numbers, and extra spaces
    - Removing stopwords
    - Tokenizing
    - Lemmatizing words
    """
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'\[.*?\]', '', text)  # Remove text inside square brackets
    text = re.sub(r'[^a-z\s]', '', text)  # Remove punctuation and special characters
    text = re.sub(r'\w*\d\w*', '', text)  # Remove words with numbers
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces

    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(text)  # Tokenize text
    filtered_words = [word for word in words if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    return ' '.join(lemmatized_words)




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ramku\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ramku\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ramku\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [141]:
strat_data["Review"] = strat_data["Review"].apply(preprocess_text)
strat_data.head(10)

,Review,Rating,Sentiment
149920,scatterbrained reference every page little ori...,1,Negative
128874,complete crap novel idea however dont work bou...,1,Negative
159208,splendid reading problem find religion u attit...,2,Positive
99436,shouldnt car game racing purchased game carsob...,1,Negative
354859,planet earth bluray world love itbeautifully p...,2,Positive
101910,doesnt hold long small car honda insight cant ...,1,Negative
349136,fantastic pricey worth every penny personally ...,2,Positive
48541,done good cant vouch quality bluray movie real...,1,Negative
96942,paint come easily sorry say throw toy away mon...,1,Negative
103884,unstable put together first anxious yr old dau...,1,Negative


In [142]:
cv = CountVectorizer(max_features = 5000)
X = cv.fit_transform(strat_data["Review"]).toarray()

## TRAIN TEST SPLIT

In [144]:
y=strat_data["Sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)


In [145]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)


LogisticRegression(random_state=0)

In [146]:
y_pred = classifier.predict(X_test)

In [147]:
accuracy_score(y_test, y_pred)

0.838